# Libraries

In [1]:
main_dir = './'
data_dir = '/home/gregory/Documents/RainPerception/' # Local Jupyter

from utilities import create_file_list, imshow_jupyter
# import carla_converter

import os

import cv2
import matplotlib.pyplot as plt
from PIL import Image

import torch
import numpy as np

import numpy as np
import os, json, cv2, random

from tqdm import tqdm

import csv

## Carla

In [4]:
data_carla_dir = data_dir + 'Carla/packaging/'

# Dir structure:
# <data_carla_dir>
#     packages2
#     packages3
#     ...

def get_carla_file_list(data_dir, packages=[], levels=[]):
    file_list = []
    
    for package in packages:
        temp_file_name_list, temp_file_path_list = create_file_list(data_dir, package)
        
        for i in range(len(temp_file_name_list)):
            file_name_split = temp_file_name_list[i].split('_')
            if len(file_name_split) == 3: # [id, type, level.png]
                file_id = file_name_split[0]
                level = file_name_split[2].replace('.png', '')
                if level in levels:
                    file_list.append((file_id, level, package))
                    
    print(f'Number of images: {len(file_list)}')
    
    return file_list

# data_carla_train_file_list = get_carla_file_list(data_carla_dir, packages=['package2', 'package3', 'package4', 'package5', 'package6', 'package7', 'package9'], levels=['H', 'M', 'S'])
# data_carla_val_file_list = get_carla_file_list(data_carla_dir, packages=['package8'], levels=['H', 'M', 'S'])

data_carla_train_file_list = get_carla_file_list(data_carla_dir, packages=['package2', 'package3', 'package4', 'package5', 'package6', 'package7', 'package8', 'package9'], levels=['H', 'M', 'S'])

# # Toy example
# data_carla_train_file_list = data_carla_train_file_list[:10]
# data_carla_val_file_list = data_carla_val_file_list[:10]

def get_carla_list_files(file_list, dataset_type='train'):
    image_list_file = open('image_list/' + dataset_type + '.txt', mode='w')
    image_csv_writer = csv.writer(image_list_file, delimiter=',')
    
    label_list_file = open('label_list/' + dataset_type + '.txt', mode='w')
    label_csv_writer = csv.writer(label_list_file, delimiter=',')
    
    for index, file in tqdm(enumerate(file_list)):
        file_id, level, package = file
        
        image_path = os.path.join('packaging/', package, file_id + '_clear.png')
        image_semantic_path = os.path.join('packaging/', package, file_id + '_semantic_train.png')
        
        image_csv_writer.writerow([image_path])
        label_csv_writer.writerow([image_semantic_path])

get_carla_list_files(data_carla_train_file_list, dataset_type='train')
# get_carla_list_files(data_carla_val_file_list, dataset_type='val')

import pickle
def get_carla_class_pixel_count(file_list, data_dir):
    train_classes = ['Unlabeled', 'Building', 'Fence', 'Pedestrian', 'Pole', 'Road', 'SideWalk', 'Vegetation', 'Vehicles', 'Wall', 'TrafficSign', 'Sky', 'TrafficLight', 'Terrain']
    num_classes = len(train_classes)
    print(f'Number of images: {len(file_list)}')
    print(f'Number of classes: {num_classes}')
    
    class_pixel_count_dict = {}
    for i in range(num_classes):
        class_pixel_count_dict[train_classes[i]] = 0
        
    class_pixel_count_per_sample_list = []
    
    for index, file in tqdm(enumerate(file_list)):
        file_id, level, package = file
        
        image_path = os.path.join('packaging/', package, file_id + '_clear.png')
        image_semantic_path = os.path.join(data_dir, package, file_id + '_semantic_train.png')
        image_semantic = cv2.imread(image_semantic_path)[:, :, 2] # HxWxC, BGR
        
        # print(image_semantic)
        # imshow_jupyter(image_semantic)
        
        class_pixel_count_dict_temp = {}
        
        for i in range(num_classes):
            class_pixel_count_dict[train_classes[i]] += np.sum(image_semantic == i)
            class_pixel_count_dict_temp[train_classes[i]] = np.sum(image_semantic == i)
            
        class_pixel_count_per_sample_list.append(class_pixel_count_dict_temp)
    
    with open('class_pixel_count_dict.pickle', 'wb') as file:
        pickle.dump(class_pixel_count_dict, file)
    
    with open('class_pixel_count_per_sample_list.pickle', 'wb') as file:
        pickle.dump(class_pixel_count_per_sample_list, file)
        
    return class_pixel_count_dict, class_pixel_count_per_sample_list

# train_classes = ['Unlabeled', 'Building', 'Fence', 'Pedestrian', 'Pole', 'Road', 'SideWalk', 'Vegetation', 'Vehicles', 'Wall', 'TrafficSign', 'Sky', 'TrafficLight', 'Terrain']
# num_classes = len(train_classes)

# class_pixel_count_carla_train_dict, class_pixel_count_per_sample_carla_train_list = get_carla_class_pixel_count(data_carla_train_file_list, data_carla_dir)
# print(class_pixel_count_carla_train_dict)

# temp_dict = {}
# for i in range(num_classes):
#     temp_dict[train_classes[i]] = 0
# for index, item in tqdm(enumerate(class_pixel_count_per_sample_carla_train_list)):
#     for i in range(num_classes):
#         # print(temp_dict[train_classes[i]])
#         temp_dict[train_classes[i]] += item[train_classes[i]]
# print(temp_dict)
# pixel_sum = sum(temp_dict.values())
# print(pixel_sum) # 2755584000 # 3836620800

Number of images: 4163


4163it [00:00, 160993.64it/s]


In [5]:
with open('class_pixel_count_dict.pickle', 'rb') as file:
    class_pixel_count_carla_train_dict = pickle.load(file)
print(class_pixel_count_carla_train_dict)
pixel_sum = sum(class_pixel_count_carla_train_dict.values())

print(pixel_sum) # 2755584000 # 3836620800
assert pixel_sum == len(data_carla_train_file_list)*1280*720

# non_ignored_sum = sum(class_pixel_count_carla_train_dict.values()) - class_pixel_count_carla_train_dict[train_classes[0]]
# print(non_ignored_sum) # 2642685163

class_freq_carla_train_dict = {}

# # Ignore class 0
# class_freq_carla_train_dict[0] = 0.0
# for i in range(1, num_classes):
#     class_freq_carla_train_dict[i] = class_pixel_count_carla_train_dict[i]/non_ignored_sum
    
# Include all
for i in range(num_classes):
    class_freq_carla_train_dict[train_classes[i]] = class_pixel_count_carla_train_dict[train_classes[i]]/pixel_sum
    
print(class_freq_carla_train_dict)
print(sum(class_freq_carla_train_dict.values())) # 1.0
with open('class_freq_dict.pickle', 'wb') as file:
    pickle.dump(class_freq_carla_train_dict, file)
    
with open('class_freq_dict.pickle', 'rb') as file:
    x = pickle.load(file)
    print(x)

FileNotFoundError: [Errno 2] No such file or directory: 'class_pixel_count_dict.pickle'

## Cityscapes

In [6]:
data_dir_cityscapes = data_dir + 'Cityscapes/leftImg8bit/train/'
data_dir_rain_addition = data_dir + 'RainAddition/train/'
# anno_dir_cityscapes = data_dir + 'Cityscapes/gtFine/train/'
anno_dir_cityscapes = data_dir + 'Cityscapes/mapped_labels/train/'

city_name_list, _ = create_file_list(data_dir_cityscapes)
print(f'Number of cities: {len(city_name_list)}')

def get_cityscapes_file_list(data_dir, cities=[], levels=[]):
    file_list = []
    
    for city in cities:
        temp_file_name_list, _ = create_file_list(data_dir, city)
        
        for file_name in temp_file_name_list:
            file_list.append((file_name, city))
                    
    print(f'Number of images: {len(file_list)}')
    
    return file_list

# data_cityscapes_train_file_list = get_cityscapes_file_list(data_dir_cityscapes, cities=city_name_list[:13], levels=['H', 'M', 'S'])
# data_cityscapes_val_file_list   = get_cityscapes_file_list(data_dir_cityscapes, cities=city_name_list[13:], levels=['H', 'M', 'S'])


data_cityscapes_train_file_list = get_cityscapes_file_list(data_dir_cityscapes, cities=city_name_list, levels=['H', 'M', 'S'])

# # Toy example
# data_cityscapes_train_file_list = data_cityscapes_train_file_list[:10]
# data_cityscapes_val_file_list = data_cityscapes_val_file_list[:10]

def get_cityscapes_list_files(file_list, dataset_type='train'):

    image_list_file = open('image_list/' + dataset_type + '.txt', mode='w')
    image_csv_writer = csv.writer(image_list_file, delimiter=',')
    
    label_list_file = open('label_list/' + dataset_type + '.txt', mode='w')
    label_csv_writer = csv.writer(label_list_file, delimiter=',')
    
    for index, file in tqdm(enumerate(file_list)):
        
        file_name, city = file

        image_path = os.path.join(city, file_name)
        
        file_name_split = file_name.split('_')
        image_id = file_name_split[0] + '_' + file_name_split[1] + '_' + file_name_split[2]
        anno_name = image_id + '_train.png' # Mapped
        image_semantic_path = os.path.join(city, anno_name)
        
        image_csv_writer.writerow([image_path])
        label_csv_writer.writerow([image_semantic_path])

get_cityscapes_list_files(data_cityscapes_train_file_list, dataset_type='train')
# get_cityscapes_list_files(data_cityscapes_val_file_list, dataset_type='val')

Number of cities: 18
Number of images: 2975


2975it [00:00, 159708.87it/s]
